In [1]:
import torch
import torch.nn as nn
import numpy as np
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.utils.data as data_utils
from ucimlrepo import fetch_ucirepo
from torch.utils.data import DataLoader
from model import WideModel
from tqdm.auto import tqdm

In [2]:
# load dataset
# pip install ucimlrepo

# fetch dataset 
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17) 
  
# data (as pandas dataframes) 
X = torch.tensor(breast_cancer_wisconsin_diagnostic.data.features.values, dtype=torch.float32)
y = breast_cancer_wisconsin_diagnostic.data.targets
y["Diagnosis"] = y["Diagnosis"].map({"M": 1, "B": 0})
y = torch.tensor(y.values, dtype=torch.float32)

train = data_utils.TensorDataset(X[:455], y[:455])
test = data_utils.TensorDataset(X[455:], y[455:])

/var/folders/jc/96rhddjx1b77w_4r1x86flk80000gn/T/ipykernel_58768/2578780564.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y["Diagnosis"] = y["Diagnosis"].map({"M": 1, "B": 0})


In [3]:

# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0,), (1,))  # Normalize with mean and std for MNIST
# ])

# mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
# mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)


In [4]:
# create dataloaders. Batch size must be 1
batch_size = 1
train_dataloader = DataLoader(train, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test, batch_size=batch_size, shuffle=True)

In [5]:
# create device
dev = "cuda" if torch.cuda.is_available() else "cpu"
print("using device", dev)

# create model
model = WideModel(input_dim=30,hidden_dim_scale = 20, output_dim=1).to(dev)

# create optimizer
lr = 1e-3
optimizer = torch.optim.SGD([p for p in model.parameters()], lr=lr)

using device cpu


In [6]:
# get linearized models:
# we reduce f(x,w) to Aw+B, where there is a different A,B per x

As = []
Bs = []
ys = []

for x,y in tqdm(train_dataloader):
    x = x.to(dev)
    
    # A = gradient matrix of logits
    A = model.flatten_gradient(x)
    
    # B = f(x,w) - A w
    B = model.forward(x) - A @ model.flatten_parameters()
    
    As.append(A.detach().cpu().numpy())
    Bs.append(B.detach().cpu().numpy())
    ys.append(y.detach().cpu().numpy())
    

  0%|          | 0/455 [00:00<?, ?it/s]